## **Knapsack Problem**

This notebook solves the 0-1 Knapsack Problem in two different ways.

1. **Greedy Algorithm:** Here we select the items in three ways. We give more importance to either weight, value or density(value/weight) of the items. All the three instances have been solved.

2. **Gurobipy:** Here we use external optimization package known as Gurobi and find te optimal selection of items with the use of binary constriants. 

In [1]:
import numpy as np
np.random.seed(1) # random number generator

In [ ]:
number_of_items = 100 
weights_mu, weights_sigma = 30, 4  # weights mean and standard deviation
weights = abs(np.around(np.random.normal(weights_mu, weights_sigma, number_of_items),2))  # item weights normally distributed
values_mu, values_sigma = 70, 10  # values mean and standard deviation
values = abs(np.around(np.random.normal(values_mu, values_sigma, number_of_items),2))  # item values normally distributed
maximum_weight = 250  # maximum weight which is allowed to select 
density = [i/j for i,j in zip(values, weights)]  # density = value/weight

**Greedy Algorithm Implementation**

In [ ]:
# Greedy Algo
input = []
for i in range(number_of_items):
  input.append([i,values[i],weights[i],density[i]])

def greedy(input, maximum_weight, parameter):
  if parameter == "weight":
    new_input = sorted(input, key = lambda x: x[2])
  elif parameter == "value":
    new_input = sorted(input, key = lambda x: x[1], reverse = True) 
  elif parameter == "density":
    new_input = sorted(input, key = lambda x: x[3], reverse = True)

  result = []
  total_value = 0
  total_weight = 0

  for i in range(len(new_input)):
    if total_weight + new_input[i][2] <= maximum_weight:
      total_value += new_input[i][1]
      total_weight += new_input[i][2]
      result.append(new_input[i])
  return result, total_value


def testGreedy(input, maximum_weight, parameter):
  result, total_value = greedy(input, maximum_weight, parameter)
  if parameter == "value":
    print('Using Greedy by Value of the item to allocate a maximum weight of', maximum_weight)
  elif parameter == "weight":
    print('Using Greedy by Weight of the item to allocate a maximum weight of', maximum_weight)
  elif parameter == "density":
    print('Using Greedy by Density(value/weight) of the item to allocate a maximum weight of', maximum_weight)
  print('Total value of items:', total_value)
  print('Number of items selected:', len(result))
  for item in range(len(result)):
    print('Item' + str(result[item][0]) + ': ' + 'Value = ' + str(result[item][1]) + ', ' + 'Weight = ' + str(result[item][2]))


def greedyTesting(input, maximum_weight):
  testGreedy(input, maximum_weight, "value")
  print('\n')
  testGreedy(input, maximum_weight, "weight")
  print('\n')
  testGreedy(input, maximum_weight, "density")

In [ ]:
greedyTesting(input, maximum_weight)

Using Greedy by Value of the item to allocate a maximum weight of 250
Total value of items: 710.3000000000002
Number of items selected: 8
Item72: Value = 95.28, Weight = 30.64
Item68: Value = 91.91, Weight = 38.74
Item53: Value = 91.87, Weight = 28.6
Item23: Value = 89.67, Weight = 32.01
Item92: Value = 89.05, Weight = 28.5
Item26: Value = 86.28, Weight = 29.51
Item76: Value = 83.31, Weight = 28.78
Item83: Value = 82.93, Weight = 31.64


Using Greedy by Weight of the item to allocate a maximum weight of 250
Total value of items: 685.3199999999999
Number of items selected: 10
Item5: Value = 71.69, Weight = 20.79
Item11: Value = 73.15, Weight = 21.76
Item75: Value = 67.74, Weight = 21.91
Item70: Value = 63.53, Weight = 24.22
Item69: Value = 51.04, Weight = 24.41
Item52: Value = 80.39, Weight = 25.43
Item36: Value = 75.21, Weight = 25.53
Item15: Value = 56.88, Weight = 25.6
Item20: Value = 69.75, Weight = 25.6
Item3: Value = 75.94, Weight = 25.71


Using Greedy by Density(value/weight) of

**Gurobipy Implementation**

In [ ]:
# Gurobipy installation and importing
%pip install -i https://pypi.gurobi.com gurobipy  
import gurobipy as gp
from gurobipy import *

Looking in indexes: https://pypi.gurobi.com


In [ ]:
# Initializing model
optimization_model = gp.Model("Knapsack Problem")

# Declare decision variables
x = optimization_model.addVars(number_of_items, vtype = GRB.BINARY, name = "Option")

# Objective function
optimization_model.setObjective(gp.quicksum(x[i]*values[i] for i in range(number_of_items)), GRB.MAXIMIZE)

# Constraint
optimization_model.addConstr(gp.quicksum(x[i]*weights[i] for i in range(number_of_items)) <= maximum_weight, name = "Weight Constraint")

# Solving the model
optimization_model.setParam('LogToConsole', 0)
optimization_model.optimize()

if optimization_model.status == GRB.OPTIMAL:
  print("\nOptimal Solution: %g" % optimization_model.objVal)
else: 
  print('Optimization Model Status %d' % optimization_model.status())

# Items selected
print("\nSelected Items:")
count = 0
for variables in optimization_model.getVars():
  if variables.x > 0.0001:
    count += 1
    print("Item" + str(variables.index) + ": " + "Value = " + str(values[variables.index]) + ", " + "Weight = " + str(weights[variables.index]))
print('\nNumber of items selected:', count)



Optimal Solution: 770.93

Selected Items:
Item3: Value = 75.94, Weight = 25.71
Item5: Value = 71.69, Weight = 20.79
Item11: Value = 73.15, Weight = 21.76
Item25: Value = 82.36, Weight = 27.27
Item36: Value = 75.21, Weight = 25.53
Item52: Value = 80.39, Weight = 25.43
Item53: Value = 91.87, Weight = 28.6
Item70: Value = 63.53, Weight = 24.22
Item75: Value = 67.74, Weight = 21.91
Item92: Value = 89.05, Weight = 28.5

Number of items selected: 10
